In [ ]:
# voice recorder3
# https://gist.github.com/arctic-hen7/5580d5452f77d4e6b206caf90f8d73c6

In [ ]:
import sounddevice
import pyaudio
import wave
import sys
import tempfile
from ctypes import *
import platform

from mixupmodel_inference import device, MixupModel_Inference

In [ ]:
# Load Mixup model once
modeldir='models'
mixup_model_desc=f'model_stage2_mix_new6c_ep32_ctc30'
mixup_modelpath=f'{modeldir}/{mixup_model_desc}.pth'
model=MixupModel_Inference(mixup_modelpath,device=device)

# Records audio directly from the microphone and then transcribes it to text using Whisper, returning that transcription.
def transcribe_directly():
    # Create a temporary file to store the recorded audio (this will be deleted once we've finished transcription)
    temp_file = tempfile.NamedTemporaryFile(suffix=".wav")

    sample_rate = 16000
    bits_per_sample = 16
    chunk_size = 1024
    audio_format = pyaudio.paInt16
    channels = 1

    def callback(in_data, frame_count, time_info, status):
        wav_file.writeframes(in_data)
        return None, pyaudio.paContinue

    # Open the wave file for writing
    wav_file = wave.open(temp_file.name, 'wb')
    wav_file.setnchannels(channels)
    wav_file.setsampwidth(bits_per_sample // 8)
    wav_file.setframerate(sample_rate)

    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Start recording audio
    stream = audio.open(format=audio_format,
                        channels=channels,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=chunk_size,
                        stream_callback=callback)

    input("Press Enter to stop recording...")
    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Close the wave file
    wav_file.close()

    # And transcribe the audio to text (suppressing warnings about running on a CPU)
    result = model.transcribe(temp_file.name, fp16=False)
    temp_file.close()

    return result["text"].strip()

In [ ]:
transcribe_directly()